<a href="https://colab.research.google.com/github/ahsaniqbal/CharacterLevelLM/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MachineLearning/"
base_dir = root_dir + 'CharLM/'

Mounted at /content/gdrive


In [4]:
import os
import os.path as osp
os.makedirs(base_dir)
os.chdir(base_dir)

OSError: ignored

In [2]:
!rm -r CharacterLevelLM
!git clone https://github.com/ahsaniqbal/CharacterLevelLM
import torch

rm: cannot remove 'CharacterLevelLM': No such file or directory
Cloning into 'CharacterLevelLM'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 60 (delta 21), reused 51 (delta 16), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [0]:
from CharacterLevelLM.Datasets.CharDataset import CharDataset, CustomCollate
from CharacterLevelLM.Models.LSTMBased import LSTMModel

In [0]:
dataset_train = CharDataset('./CharacterLevelLM/data.txt', 100, train_size=0.95)
dataset_test = CharDataset('./CharacterLevelLM/data.txt', 100, is_train=False, train_size=0.95)

In [5]:
dataset_test.get_vocab_size()

62

In [0]:
model = LSTMModel(dataset_train.get_vocab_size(), 128, dataset_test.get_vocab_size())

In [0]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset_train, batch_size=50, shuffle=True, collate_fn=CustomCollate())
loader_test = DataLoader(dataset_test, batch_size=50, shuffle=True, collate_fn=CustomCollate())

In [0]:
from torch.optim import Adam
optimizer = Adam(model.parameters())

In [0]:
from torch import nn
criterion = nn.CrossEntropyLoss(reduction = 'none')

In [0]:
import numpy as np
best_loss = np.inf

In [12]:
model = model.cuda()
for i in range(2000):
  model.train()
  for data in loader_train:
    optimizer.zero_grad()
    X, Y, mask = data
    X, Y, mask = X.cuda(), Y.cuda(), mask.cuda()
    
    Y = Y.view(Y.shape[0] * Y.shape[1])
    mask = mask.view(mask.shape[0] * mask.shape[1])

    loss = (criterion(model(X), Y) * mask).mean()
    loss.backward()
    optimizer.step()
  model.eval()

  test_loss = 0 
  with torch.no_grad():
    for idx, data in enumerate(loader_test):
      X, Y, mask = data
      X, Y, mask = X.cuda(), Y.cuda(), mask.cuda()

      Y = Y.view(Y.shape[0] * Y.shape[1])
      mask = mask.view(mask.shape[0] * mask.shape[1])

      test_loss *= idx
      test_loss += (criterion(model(X), Y) * mask).mean().item()
      test_loss /= (idx + 1)
  print(test_loss)
  if test_loss < best_loss:
    best_loss = test_loss
    torch.save({'state_dict': model.state_dict()}, base_dir + '/model.pth')


3.289442539215088


FileNotFoundError: ignored